In [4]:
import os
import sys
import torch
import torchvision
import numpy as np
import torchvision.transforms as transforms
import einops
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

if "x-perceiver" not in os.listdir():
    os.chdir("/Users/konsti/Documents/repos/phd/x-perceiver/")

%reload_ext autoreload
%autoreload 2

In [6]:
from x_perceiver.utils import Config
from x_perceiver.etl import TCGADataset
from x_perceiver.train import train_loop

### Load data

In [57]:
# Molecular data
config = Config("config/main.yml").read()
blca = TCGADataset("blca", config, filter_omic=False)

Dataset: BLCA
Molecular data shape: (437, 2191)
Total slides available: 25
Molecular/Slide match: 25/437
Slide level count: 4
Slide level dimensions: ((98056, 76793), (24514, 19198), (6128, 4799), (3064, 2399))
Slide resize dimensions: w: 1920, h: 1664
Sources selected: ['omic', 'slides']
Censored share: 0.54


In [58]:
blca.omic_df.shape

(437, 2191)

In [97]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
def train_random_forest(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    maj_class_clf_test = np.unique(ar=y_test, return_counts=True)[1][-1] / len(y_test)
    # gradient boosting
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=2, min_samples_split=2, random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(y_pred)
    print(maj_class_clf_test)
    # print(majority_class_clf_test)
    return accuracy_score(y_test, y_pred)

In [93]:
X = blca.features.values
y = blca.omic_df["y_disc"].values

In [98]:
train_random_forest(X, y)

[3 1 0 3 3 3 3 3 3 3 1 0 2 1 2 1 1 0 0 1 2 1 0 3 3 3 3 3 1 3 3 0 0 3 3 1 2
 0 2 3 1 3 3 2 2 2 0 3 1 3 3 2 0 2 2 1 2 2 2 3 2 2 2 2 2 3 3 3 3 3 2 3 0 2
 1 3 3 3 2 2 0 3 0 1 2 3 3 3]
0.42045454545454547


0.3181818181818182

In [103]:
unique, class_counts = np.unique(ar=blca.y_disc, return_counts=True)

In [104]:
class_counts

array([ 72,  83, 109, 173])